In [1]:
using TSSOS
using LinearAlgebra
using QuantumOptics
using DynamicPolynomials
using QuadGK
using JuMP
using Random
using NLopt

## Define the quantum system

In [2]:
# We look at a typical Hamiltonian

#Drift Hamiltonian

H0 = [
    0 0 0;
    0 0.515916 0;
    0 0 1
];

H0 ./= norm(H0, Inf)

# Control Hamiltonian
V = [
    0 0.707107 0;
    0.707107 0 1;
    0 1 0
]

V ./= norm(V, Inf);

In [3]:
H0*V-V*H0

3×3 Matrix{Float64}:
 0.0       -0.364808   0.0
 0.364808   0.0       -0.484084
 0.0        0.484084   0.0

## Useful functions with polynomials

In [4]:
function ∫(p::AbstractPolynomial, x::PolyVar, x_lower, x_upper)
    
    # get the index of the variable of integration
    ind_x = indexin([x], variables(p))[1]
        
    if isnothing(ind_x)
        # integration valuable is not found among vars
        return p * (x_upper - x_lower)
    end
    
    # get the indefinite integral
    int_p = sum(
        term * x * 1 // (exponents(term)[ind_x] + 1) for term in terms(p)
        init = 0 * x
    )
            
    # get the definite integral
    subs(int_p, x=>x_upper) - subs(int_p, x=>x_lower)
end

function ∫(M::AbstractMatrix, x::PolyVar, x_lower, x_upper)
   map(z -> ∫(z, x, x_lower, x_upper), M) 
end

function real_poly(p::Polynomial)
    #=
    Real part of the polynomial
    =#
    sum(
        real(c) * m for (c, m) in zip(coefficients(p), monomials(p))# if ~isapproxzero(abs(c))
    )
end

function square_frobenius_norm(M::AbstractArray)
    #=
    Square of the Frobenius norm of a matrix
    =#
    real_poly(sum(z' * z for z in M))
end

function square_frobenius_norm2(M::AbstractArray)
    #=
    Square of the Frobenius norm of a matrix
    =#
    sum(z' * z for z in M)
end

square_frobenius_norm2 (generic function with 1 method)

## Magnus expansion

In [5]:
@polyvar x[1:3]
@polyvar t[1:3]

# final time
const T = 0.5

function u(t, x)
    # the polynomial shape for control
    sum(x[n] * t^(n - 1) for n = 1:length(x))
end

function A(t, x,)
    #=
    The generator of motion entering the Magnus expansion
    =#
    (H0 + V * u(t, x)) / im
end

function commutator(a, b)
    a * b - b * a
end 

# get the partial sum of the Magnus expansion
A₁ = A(t[1], x)
A₂ = A(t[2], x)

Ω = ∫(A₁, t[1], 0, T);

# 2nd term in the Magnus expansion
Ω .+= 1//2 * ∫(∫(
    commutator(A₁, A₂), 
    t[2], 0, t[1]), 
    t[1], 0, T
);

# 3nd term in the Magnus expansion

A₃ = A(t[3], x)

Ω .+= 1//6 * ∫(∫(∫(
    commutator(A₁, commutator(A₂, A₃)) + commutator(commutator(A₁, A₂), A₃),
    t[3], 0, t[2]),
    t[2], 0, t[1]),
    t[1], 0, T
);


In [6]:
println(Ω)

Polynomial{true}[(0.0 + 4.4784402716960125e-5im)x₁x₃ + (0.0 - 6.717660407544019e-5im)x₂² + (0.0 - 5.5980503396200154e-5im)x₂x₃ + (0.0 - 1.3328691284809559e-5im)x₃² (0.0 - 0.3535535im)x₁ + (-0.0038000814063750003 - 0.088388375im)x₂ + (-0.0019000407031875004 - 0.02944645397667624im)x₃ (0.0 + 1.953874134027782e-6im)x₁x₃ + (0.0 - 2.930811201041674e-6im)x₂² + (0.0 - 2.442342667534728e-6im)x₂x₃ + (0.0 - 5.8151015893684e-7im)x₃²; (0.0 - 0.3535535im)x₁ + (0.0038000814063750003 - 0.088388375im)x₂ + (0.0019000407031875004 - 0.02944645397667624im)x₃ (0.0 + 7.228014483236696e-20im)x₁x₂ + (0.0 + 3.925795839415097e-5im)x₁x₃ + (0.0 - 5.888693759122647e-5im)x₂² + (0.0 - 4.907244799268873e-5im)x₂x₃ + (0.0 - 1.168391618873541e-5im)x₃² + (0.0 - 0.257958im) (0.0 - 0.5im)x₁ + (-0.005042541666666666 - 0.125im)x₂ + (-0.0025212708333333333 - 0.04164632488549861im)x₃; (0.0 + 1.953874134027782e-6im)x₁x₃ + (0.0 - 2.930811201041674e-6im)x₂² + (0.0 - 2.442342667534728e-6im)x₂x₃ + (0.0 - 5.8151015893684e-7im)x₃² (0

## Get random unitary target

In [7]:
Random.seed!(65)
exact_x = -1 .+ 2 * rand(3)

function get_unitary(x::AbstractArray)
    #=
    Get the unitary given the coefficients for the polynomial control
    =#
    basis = NLevelBasis(size(H0)[1])

    𝓗₀ = DenseOperator(basis, basis, H0)
    𝓥 = DenseOperator(basis, basis, V)

    H = LazySum([1., u(0, x)], [𝓗₀, 𝓥])
        
    function 𝓗(t, psi)
        H.factors[2] = u(t, x)
        return H
    end

    _, 𝓤 = timeevolution.schroedinger_dynamic([0, T], identityoperator(basis, basis), 𝓗)
    
    return Matrix(𝓤[2].data)
end

get_unitary (generic function with 1 method)

In [8]:
exact_x

3-element Vector{Float64}:
 -0.1874836569137377
 -0.8380629706479825
 -0.3946330275188754

In [9]:
U_target=get_unitary(exact_x)

3×3 Matrix{ComplexF64}:
   0.988573+0.000915344im  0.0229879+0.1481im    -0.0154305+0.00481831im
  0.0152846+0.149092im       0.93396-0.245578im   0.0832998+0.19496im
 -0.0158348+0.00324486im   0.0737072+0.198785im    0.856689-0.469973im

In [10]:
F= eigen(U_target)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
3-element Vector{ComplexF64}:
 0.8048207561278221 - 0.5935179319537021im
 0.9781662887906366 - 0.20782374776455456im
 0.9962339862643255 + 0.08670550462022175im
vectors:
3×3 Matrix{ComplexF64}:
  0.126429-0.00636229im  -0.504082+0.0130641im  0.854228+0.0im
 -0.528858+0.0128848im    0.694153+0.0im         0.48799+0.0126479im
  0.839117+0.0im          0.513542+0.0125125im  0.178658+0.00898771im

In [11]:
X = F.vectors
L = Diagonal(F.values)

3×3 Diagonal{ComplexF64, Vector{ComplexF64}}:
 0.804821-0.593518im           ⋅                    ⋅    
          ⋅           0.978166-0.207824im           ⋅    
          ⋅                    ⋅           0.996234+0.0867055im

In [12]:
X*L*inv(X)

3×3 Matrix{ComplexF64}:
   0.988573+0.000915344im  0.0229879+0.1481im    -0.0154305+0.00481831im
  0.0152846+0.149092im       0.93396-0.245578im   0.0832998+0.19496im
 -0.0158348+0.00324486im   0.0737072+0.198785im    0.856689-0.469973im

In [17]:
n= 0
theta_values = [log(λ) for λ in F.values]
K = Diagonal(theta_values + 2*pi*im*n*[1,1,1])

3×3 Diagonal{ComplexF64, Vector{ComplexF64}}:
 -7.47762e-9-0.635423im               ⋅                       ⋅    
             ⋅           -6.69324e-10-0.20935im               ⋅    
             ⋅                        ⋅          -4.02222e-11+0.0868145im

## TSSOS

In [32]:
obj = square_frobenius_norm(
        Ω - X*K*inv(X)
    );

# Get the global minimum via TSSOS library
opt,sol,data = tssos_first(obj, variables(obj); QUIET = true, solution = true);

previous_sol = sol
previous_opt = opt
    
    while ~isnothing(sol)
        previous_sol = sol
        previous_opt = opt
            
        opt,sol,data = tssos_higher!(data; QUIET = true, solution = true)
    end
println(previous_sol)

*********************************** TSSOS ***********************************
TSSOS is launching...
optimum = 2.8689630976624563e-8
Global optimality certified with relative optimality gap 0.000003%!
No higher TS step of the TSSOS hierarchy!
[-0.18898455444753834, -0.816553648388228, -0.4408514842701129]


In [33]:
println(previous_opt)

2.8689630976624563e-8


In [26]:
println(obj)

5.224419336987944e-39*x[1]^2*x[2]^2 + 5.675141837104536e-24*x[1]^2*x[2]*x[3] + 1.0617583733385394e-8*x[1]^2*x[3]^2 - 8.512712755656808e-24*x[1]*x[2]^3 - 3.18527512001562e-8*x[1]*x[2]^2*x[3] - 2.6543959333463494e-8*x[1]*x[2]*x[3]^2 - 6.319990317491306e-9*x[1]*x[3]^3 + 2.388956340011715e-8*x[2]^4 + 3.981593900019525e-8*x[2]^3*x[3] + 2.6069960059651652e-8*x[2]^2*x[3]^2 + 7.899987896864136e-9*x[2]*x[3]^3 + 9.404747496266827e-10*x[3]^4 + 0.7500001547245001*x[1]^2 + 0.37500007736225005*x[1]*x[2] + 0.12493626794835613*x[1]*x[3] + 0.04695469929080999*x[2]^2 + 0.03131375660761778*x[2]*x[3] + 0.005222944838285926*x[3]^2 + 0.6448029335963948*x[1] + 0.1613662312875644*x[2] + 0.053788906056044694*x[3] + 0.13867614414243581


## Check how close unitary is

In [27]:
println(exact_x, previous_sol)

[-0.1874836569137377, -0.8380629706479825, -0.3946330275188754][-0.18898455444753834, -0.816553648388228, -0.4408514842701129]


In [28]:
get_unitary(previous_sol)

3×3 Matrix{ComplexF64}:
   0.988574+0.000915874im  0.0229926+0.14809im   -0.0154282+0.004819im
  0.0152774+0.149084im      0.933964-0.245579im   0.0833026+0.194945im
 -0.0158331+0.0032432im     0.073695+0.198776im    0.856691-0.469975im

In [29]:
get_unitary(exact_x)

3×3 Matrix{ComplexF64}:
   0.988573+0.000915344im  0.0229879+0.1481im    -0.0154305+0.00481831im
  0.0152846+0.149092im       0.93396-0.245578im   0.0832998+0.19496im
 -0.0158348+0.00324486im   0.0737072+0.198785im    0.856689-0.469973im

In [30]:
square_frobenius_norm2(get_unitary(previous_sol)-get_unitary(exact_x))

7.441337008412143e-10 + 0.0im

In [31]:
println(opt)

nothing
